# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [1]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [10]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [3]:
boston[!, :LogMedV] = log.(boston[!, :MedV])

506-element Array{Float64,1}:
 3.1780538303479458
 3.0726933146901194
 3.5467396869528134
 3.5085558999826545
 3.5890591188317256
 3.3568971227655755
 3.131136910560194
 3.299533727885655
 2.803360380906535
 2.9391619220655967
 2.70805020110221
 2.9391619220655967
 3.077312260546414
 ⋮
 3.1986731175506815
 3.139832617527748
 2.9806186357439426
 2.9069010598473755
 3.054001181677967
 2.8622008809294686
 2.8213788864092133
 3.109060958860994
 3.0252910757955354
 3.173878458937465
 3.091042453358316
 2.4765384001174837

In [4]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [5]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,2.01019,0.0,19.58,0,0.605,7.929,96.2,2.0459,5,403
2,7.52601,0.0,18.1,0,0.713,6.417,98.3,2.185,24,666
3,0.15098,0.0,10.01,0,0.547,6.021,82.6,2.7474,6,432
4,10.6718,0.0,18.1,0,0.74,6.459,94.8,1.9879,24,666
5,0.0536,21.0,5.64,0,0.439,6.511,21.1,6.8147,4,243
6,14.4383,0.0,18.1,0,0.597,6.852,100.0,1.4655,24,666
7,0.02875,28.0,15.04,0,0.464,6.211,28.9,3.6659,4,270
8,16.8118,0.0,18.1,0,0.7,5.277,98.1,1.4261,24,666
9,1.34284,0.0,19.58,0,0.605,6.066,100.0,1.7573,5,403


In [6]:
model = lm(@formula(LogMedV ~ Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

LogMedV ~ 1 + Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                 Estimate   Std. Error    t value  Pr(>|t|)     Lower 95%     Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)   4.27954      0.222237      19.2566     <1e-57   3.84262       4.71646
Crim         -0.0079538    0.00150794    -5.27461    <1e-6   -0.0109184    -0.00498918
Chas          0.109353     0.0387103      2.8249     0.0050   0.0332483     0.185457
NOx          -0.698811     0.153786      -4.54405    <1e-5   -1.00115      -0.396467
Rm            0.073531     0.0175125      4.19877    <1e-4    0.0391013     0.107961
Dis          -0.0471648    0.00723896    -6.

In [7]:
exp.(predict(model, test))

101-element Array{Float64,1}:
 36.52870776467934
 15.008053006401026
 23.27196806300342
 11.170923596924316
 28.435792426770046
 15.343666861945099
 28.593946341078155
  9.677294165853283
 29.55310756064864
  8.411008177811206
 28.567732014583186
 24.499002592074113
 35.200392988711556
  ⋮
 32.49513585227245
 31.790079084569108
 27.98744110897146
 21.28740597919645
 29.245837094896242
 34.63602158623184
 32.90922541369443
 17.32776314061427
 27.311430495928157
 14.431169581672968
 17.624725484209815
 19.621966062356535

In [8]:
adjr²(model)

0.783003751846593